# ELECTRIC VEHICLE CHARGE SCHEDULING 
Objective: Minimize the cost incurred by the charging station.

In [65]:
# Import libraries
import numpy as np
import copy
import random
import fileinput

In [66]:
# genrand = True: Generate a random float value between provided constraints.
# genrand = False: Simply sends the already generated value in rand_vals array.
def fRand (fMin,fMax, h, i, gen_rand, rand_vals, tot_splits, split_no):
  if (gen_rand):
    return random.uniform(fMin + (fMax-fMin)*split_no/tot_splits, fMin + (fMax-fMin)*(split_no + 1)/tot_splits)
  else:
    return rand_vals[i][h]

In [67]:
# Stores and finds the delta E values of each of the EVs in a given time slot and check whether the rand_vals array is valid.
def cal_deltaE (h, EV, deltaT, n, LR, gen_rand, rand_vals, tot_splits, split_lst, pop_LR): 
  for i in range(1, n+1, 1):
    if (h*deltaT > EV[i]['T_fin'] or h*deltaT <= EV[i]['T_init']): 
      EV[i]['deltaE_const'] = EV[i]['deltaE'] = 0
    
    elif (EV[i]['type'] == False):
      left = max(EV[i]['MinPwLmt']*deltaT, EV[i]['E_fin'] - EV[i]['E_curr'])
      right = min(0.0, EV[i]['E_fin'] - EV[i]['E_curr'] - EV[i]['MinPwLmt']*(EV[i]['T_fin'] - h*deltaT))
      if (h == 1 and pop_LR):
        LR[i][0] = left
        LR[i][1] = right
  
      if (h == 1):
        EV[i]['deltaE_const'] = EV[i]['deltaE'] = fRand(LR[i][0], LR[i][1], h, i, gen_rand, rand_vals, tot_splits, split_lst[i])
      else:
        EV[i]['deltaE_const'] = EV[i]['deltaE'] = fRand(left, right, h, i, gen_rand, rand_vals, tot_splits, split_lst[i])
      
    else:
      left = max(max(EV[i]['MaxCap']-EV[i]['E_curr'], EV[i]['MinPwLmt']*deltaT), EV[i]['E_fin'] - EV[i]['E_curr'] - EV[i]['MaxPwLmt']*(EV[i]['T_fin'] - h*deltaT))
      right = min(min(-1*EV[i]['E_curr'], EV[i]['MaxPwLmt']*deltaT), EV[i]['E_fin'] - EV[i]['E_curr'] - EV[i]['MinPwLmt']*(EV[i]['T_fin'] - h*deltaT))
      if (h == 1 and pop_LR):
        LR[i][0] = left
        LR[i][1] = right
  
      if (h == 1):
        EV[i]['deltaE_const'] = EV[i]['deltaE'] = fRand(LR[i][0], LR[i][1], h, i, gen_rand, rand_vals, tot_splits, split_lst[i])
      else:
        EV[i]['deltaE_const'] = EV[i]['deltaE'] = fRand(left, right, h, i, gen_rand, rand_vals, tot_splits, split_lst[i])

  return EV, LR

In [68]:
# Find energy drawn from grid in one time slot.
# Performs energy exchange between the EVs using prioritization algorithm
# Updates Current energy and EPT values of the EVs accordingly.
def cal_E_g(h, EV, deltaT, n, P_PV, C_g, len): 
  dC_EVs = np.empty((0, 2), dtype=[('energy_cost', np.float64), ('index', np.int64)])
  dC_EVs = np.append(dC_EVs, np.array([tuple([0,0])], dtype=dC_EVs.dtype))
  dC_EVs = np.append(dC_EVs, np.array([tuple([1000000000000,n+1])], dtype=dC_EVs.dtype))

  C_EVs = np.empty((0, 2), dtype=[('dep_time', np.float64), ('index', np.int64)])

  for i in range(1, n+1, 1):
    if (h*deltaT <= EV[i]['T_fin'] and h*deltaT > EV[i]['T_init']):
      if (EV[i]['type'] == True and EV[i]['deltaE'] > 0):
        dC_EVs = np.append(dC_EVs, np.array([tuple([EV[i]['EPT'][h-1], i])], dtype=dC_EVs.dtype))
        EV[i]['EPT'][h] = EV[i]['EPT'][h-1]
      else:
        if (EV[i]['type'] == False):
          C_EVs = np.append(C_EVs, np.array([tuple([len+1, i])], dtype=C_EVs.dtype))
        else:
          C_EVs = np.append(C_EVs, np.array([tuple([EV[i]['T_fin'], i])], dtype=C_EVs.dtype))

  dC_EVs = np.sort(dC_EVs, order=['energy_cost'])
  C_EVs = np.sort(C_EVs, order=['dep_time'])[::-1]

  j = 0
  for i in range(1, n+1, 1):
    if (EV[i]['type'] == True):
      EV[i]['C_curr'] = -1*EV[i]['EPT'][h-1]*EV[i]['E_curr']

  E_PV_curr = P_PV[h]*deltaT
  E_g_curr = 0;

  for i in range(0, C_EVs.size, 1):
    C_EV = C_EVs[i]['index']

    while ((dC_EVs[j]['index'] == 0 and -EV[C_EV]['deltaE'] > E_PV_curr) or (dC_EVs[j]['index'] != 0 and dC_EVs[j]['index'] != (n+1) and -EV[C_EV]['deltaE'] > EV[dC_EVs[j]['index']]['deltaE'])):
      if (dC_EVs[j]['index'] == 0):
        EV[C_EV]['E_curr'] += -E_PV_curr
        EV[C_EV]['deltaE'] += E_PV_curr
        E_PV_curr = 0
      else:
        EV[C_EV]['E_curr'] += -EV[dC_EVs[j]['index']]['deltaE']
        if (EV[C_EV]['type'] == True):
          EV[C_EV]['C_curr'] += EV[dC_EVs[j]['index']]['EPT'][h-1]*EV[dC_EVs[j]['index']]['deltaE']
        EV[C_EV]['deltaE'] += EV[dC_EVs[j]['index']]['deltaE']
        EV[dC_EVs[j]['index']]['E_curr'] += EV[dC_EVs[j]['index']]['deltaE']
        EV[dC_EVs[j]['index']]['deltaE'] = 0
      j += 1

    if (dC_EVs[j]['index'] == 0):
      EV[C_EV]['E_curr'] += EV[C_EV]['deltaE']
      E_PV_curr += EV[C_EV]['deltaE']
      EV[C_EV]['deltaE'] = 0

    elif (dC_EVs[j]['index'] == n+1):
      EV[C_EV]['E_curr'] += EV[C_EV]['deltaE']
      if (EV[C_EV]['type'] == True):
        EV[C_EV]['C_curr'] += C_g[h]*(-1*EV[C_EV]['deltaE'])
      E_g_curr += -1*EV[C_EV]['deltaE']
      EV[C_EV]['deltaE'] = 0  
    else:
      EV[C_EV]['E_curr'] += EV[C_EV]['deltaE']
      if (EV[C_EV]['type'] == True):
        EV[C_EV]['C_curr'] += EV[dC_EVs[j]['index']]['EPT'][h-1]*(-1*EV[C_EV]['deltaE'])
    
      EV[dC_EVs[j]['index']]['E_curr'] += -1*EV[C_EV]['deltaE']
      EV[dC_EVs[j]['index']]['deltaE'] += EV[C_EV]['deltaE']    
      EV[C_EV]['deltaE'] = 0

  for i in range(1, n+1, 1):
    if (h*deltaT <= EV[i]['T_fin'] and h*deltaT > EV[i]['T_init']):
      if (EV[i]['type'] == True and EV[i]['deltaE'] > 0):
        EV[i]['E_curr'] += EV[i]['deltaE']
        EV[i]['deltaE'] = 0
  
  for i in range(1, n+1, 1):
    if (h*deltaT <= EV[i]['T_fin'] and h*deltaT > EV[i]['T_init']):
      if (EV[i]['type'] == True and EV[i]['deltaE_const'] < 0):
        EV[i]['EPT'][h] = EV[i]['C_curr']/((-1)*EV[i]['E_curr'])

  return E_g_curr, EV 

In [69]:
# Taking input from file and storing data in specific variables.
data = []
with fileinput.input(files = ('sample_data/h_16.txt')) as f:
    for line in f:
        for word in line.split():        
            data.append(word)

word = 0
EV = np.zeros(100, 
       dtype=[('type', np.bool_), ('SOC_init', np.float64), ('SOC_fin', np.float64), ('MaxCap', np.float64), ('MaxPwLmt', np.float64), ('MinPwLmt', np.float64), ('E_init', np.float64), ('E_fin', np.float64), ('E_curr', np.float64), ('TI', np.float64), ('T_init', np.float64), ('T_fin', np.float64), ('deltaE', np.float64), ('deltaE_const', np.float64), ('C_curr', np.float64), ('EPT', np.float64, 1000)])

new_arr = np.zeros(1000, dtype = np.float64)
C_g = copy.deepcopy(new_arr)
P_PV = copy.deepcopy(new_arr)
E_g = copy.deepcopy(new_arr)

H = float(data[word])
word += 1
print('Enter the length of the day: ' + str(H))

deltaT = float(data[word])
word += 1
print('Enter the length of one time unit: ' + str(deltaT))
l = (int)(H//deltaT)  
H = l*deltaT;
i = 0

print('Enter the grid prices over the whole day (' + str(l) + ' space separated no.s): ')
for i in range(1,l+1,1):
  print(data[word] + " ", end = '')
  C_g[i] = float(data[word])
  word += 1

print('\nEnter the photo voltaic power generated over the whole day (' + str(l) + ' space separated no.s): ')
for i in range(1,l+1,1):
  print(data[word] + " ", end = '')
  P_PV[i] = float(data[word])
  word += 1

n = int(data[word])
word += 1
print("\nEnter number of EVs: " + str(n))
for i in range(1, n+1, 1):
  print("\nEnter details of EV" + str(i) + " :- \n")
  ev_type = data[word]
  word += 1
  print("Enter the type of EV (VV/GV): " + ev_type)
  if (ev_type == "GV"):
    EV[i]['type'] = False
  else:
    EV[i]['type'] = True
  
  EV[i]['SOC_init'] = float(data[word])
  word += 1
  print("Enter initial SOC of the EV: " + str(EV[i]['SOC_init']))
  EV[i]['SOC_fin'] = float(data[word])
  word += 1
  print("Enter final SOC of the EV: " + str(EV[i]['SOC_fin']))
  EV[i]['MaxCap'] = float(data[word])
  word += 1
  print("Enter the max capacity of the EV: " + str(EV[i]['MaxCap']))

  EV[i]['MaxCap'] = -1*EV[i]['MaxCap']
  EV[i]['MinPwLmt'] = float(data[word])
  word += 1
  print("Enter the min power limit of the EV [-ve]: " + str(EV[i]['MinPwLmt']))
  EV[i]['MaxPwLmt'] = float(data[word])
  word += 1
  print("Enter the max power limit of the EV (0 for GV)[+ve]: " + str(EV[i]['MaxPwLmt']))
  EV[i]['TI'] = float(data[word])
  word += 1
  print("Enter the time interval of stay of EV: " + str(EV[i]['TI']))
  EV[i]['E_fin'] = (EV[i]['SOC_fin']*EV[i]['MaxCap'])/100
  EV[i]['E_curr'] = EV[i]['E_init'] = (EV[i]['SOC_init']*EV[i]['MaxCap'])/100
  while (EV[i]['TI']*EV[i]['MinPwLmt'] > EV[i]['E_fin'] - EV[i]['E_init']):
    EV[i]['TI'] = float(input("Invalid time interval of stay of EV.\nEnter a larger time interval: "))   

  EV[i]['T_init'] = float(data[word])
  word += 1
  print("Enter arrival time of the EV: " + str(EV[i]['T_init']))
  EV[i]['T_fin'] = EV[i]['T_init'] + EV[i]['TI']

f = float(data[word])
word += 1
print("Enter the fraction of charging price to give to the EV owners: " + str(f))

I = int(data[word])
word += 1
print("\n\nEnter the number of generations for the Heuristic Algorithm: " + str(I))

tot_splits = int(data[word])
word += 1
print("\n\nEnter the number of splits for the Heuristic Algorithm: " + str(tot_splits))


Enter the length of the day: 16.0
Enter the length of one time unit: 1.0
Enter the grid prices over the whole day (16 space separated no.s): 
1 4 4 3 2 4 1 4 4 3 2 4 4 3 2 4 
Enter the photo voltaic power generated over the whole day (16 space separated no.s): 
1 2 3 4 5 3 1 2 3 4 5 3 3 4 5 3 
Enter number of EVs: 5

Enter details of EV1 :- 

Enter the type of EV (VV/GV): GV
Enter initial SOC of the EV: 20.0
Enter final SOC of the EV: 70.0
Enter the max capacity of the EV: 100.0
Enter the min power limit of the EV [-ve]: -10.0
Enter the max power limit of the EV (0 for GV)[+ve]: 0.0
Enter the time interval of stay of EV: 14.0
Enter arrival time of the EV: 0.0

Enter details of EV2 :- 

Enter the type of EV (VV/GV): GV
Enter initial SOC of the EV: 30.0
Enter final SOC of the EV: 60.0
Enter the max capacity of the EV: 100.0
Enter the min power limit of the EV [-ve]: -15.0
Enter the max power limit of the EV (0 for GV)[+ve]: 0.0
Enter the time interval of stay of EV: 5.0
Enter arrival tim

In [70]:
# Make all split combinations
def make_split_combs(tot_splits, n, split_combs, comb):
  if (n == 0):
    split_combs.append(comb)
    return
  else:
    for i in range(tot_splits):
      arr = copy.deepcopy(comb)
      arr.append(i)
      make_split_combs(tot_splits, n-1, split_combs, arr)

In [71]:
# split_combs = tot_splits^n x (n+1)
split_combs = []
make_split_combs(tot_splits, n, split_combs, [0])

In [72]:
LR = np.zeros(shape=(100,2), dtype = np.float64)
rand_vals = np.zeros(shape=(100,1000), dtype = np.float64)

for i in range(1, n+1, 1):
  EV[i]['E_curr'] = EV[i]['E_init']
  EV[i]['deltaE'] = EV[i]['deltaE_const'] = 0

for h in range(1, l+1, 1):
  split_lst = (n+1)*[0]
  EV, LR = cal_deltaE(h, EV, deltaT, n, LR, True, rand_vals, 1, split_lst, True)
  E_g_curr, EV = cal_E_g(h, EV, deltaT, n, P_PV, C_g, l)

overall_best_cost = 100000000000000
while(I):
  best_cost = 100000000000000
  best_comb = []
  for comb in split_combs:
    tot_cost = 0
    for i in range(1, n+1, 1):
      EV[i]['E_curr'] = EV[i]['E_init']
      EV[i]['deltaE'] = EV[i]['deltaE_const'] = 0
    for h in range(1, l+1, 1):
      if (h == 1):
        split_lst = comb
        splits = tot_splits
      else:
        split_lst = (n+1)*[0]
        splits = 1
      EV, LR = cal_deltaE(h, EV, deltaT, n, LR, True, rand_vals, splits, split_lst, False)
      E_g_curr, EV = cal_E_g(h, EV, deltaT, n, P_PV, C_g, l)
      E_g[h] = E_g_curr
      curr_g_price = C_g[h]*E_g_curr
      tot_cost += curr_g_price

      for i in range(1, n+1, 1):
        if (h*deltaT <= EV[i]['T_fin'] and h*deltaT > EV[i]['T_init']):
          if (EV[i]['type'] == True):
            tot_cost += max((0, (EV[i]['EPT'][h-1]*EV[i]['deltaE_const']*f)))
            
    if (tot_cost < best_cost):
      best_comb = comb
      best_cost = tot_cost
  
  for i in range(1, n+1):
    fMin = LR[i][0]
    fMax = LR[i][1]
    LR[i][0] = fMin + (fMax-fMin)*best_comb[i]/tot_splits
    LR[i][1] = fMin + (fMax-fMin)*(best_comb[i] + 1)/tot_splits

  if (overall_best_cost > best_cost):
      overall_best_cost = best_cost
  
  I -= 1

print("\nMin cost to the Charging Station using HA: " + str(overall_best_cost))


Min cost to the Charging Station using HA: 538.4474914798265
